In [15]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from sklearn.cluster import KMeans
import pickle

from scipy.spatial.distance import cdist

In [16]:
def corvert_to_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def SIFT(image):
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(image, None)
    return  descriptors

In [17]:
def kmeans_bow(all_descriptors, num_clusters):

    bow_dict = []
    kmeans = KMeans(n_clusters=num_clusters, verbose = 1).fit(all_descriptors)
    bow_dict = kmeans.cluster_centers_
    
    return bow_dict

In [18]:
path = 'D:\PTIT\CSDLDPT\Multimedia-Database-System\Mix'

features = []
image_features = []
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        image = cv2.imread(os.path.join(dirname, filename))
        gray = corvert_to_gray(image)
        sift = SIFT(gray)
        image_features.append(sift)
        for i in sift:
            features.append(i)

In [19]:
num_clusters = 50
file_path = os.path.join(os.getcwd(), 'bow_dictionary.pkl')
if not os.path.isfile(file_path):
    BoW = kmeans_bow(features, num_clusters)
    with open(file_path, 'wb') as file:
        pickle.dump(BoW, file)
else:
    with open(file_path, 'rb') as file:
            BoW = pickle.load(file)

In [20]:
def create_features_bow(single_image_descriptors, BoW, num_clusters):
    feature = np.array([0] * num_clusters)
    if single_image_descriptors is not None:
        distance = cdist(single_image_descriptors, BoW, metric='euclidean')
        argmin = np.argmin(distance, axis=1)
        for j in argmin:
            feature[j] += 1
    return feature

In [25]:
tmp = 0

features = []


for i in image_features:
    feature = create_features_bow(i, BoW, num_clusters)
    features.append(feature)

tp = (features)

print(tp)



[array([1, 3, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]), array([1, 5, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]), array([1, 4, 0, 0, 2, 1, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1, 0,
       0, 1, 4, 0, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 1, 1, 1, 0, 0, 7, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 4, 0, 0, 0, 8, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0]), array([1, 5, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 4, 0, 0, 1, 5, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0]), array([0, 5, 0, 0, 0, 0, 1, 0, 1, 5, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0,
       0, 0, 9, 0, 1, 0, 7, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 0, 